In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.linear_model import LinearRegression, Lasso, LassoCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler, PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error

In [2]:
df = pd.read_csv('datasets/train.csv', keep_default_na=False, na_values=[''])
df = df.rename(str.lower, axis=1)
df.columns = [name.replace(' ', '_') if ' ' in name else name for name in df.columns]

# Model 2
df['age'] = [df['year_built'].max()-year for year in df['year_built']]
df['age*overall_qual'] = df['age']*df['overall_qual']

# Model 3
kitchen_qual_order = {'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4}
df['kitchen_qual_order'] = [kitchen_qual_order[n] for n in df['kitchen_qual']]

# Model 4
df['gr_liv_area/totrms_abvgrd'] = df['gr_liv_area']/df['totrms_abvgrd']
df['since_remod/add'] = [2010-year for year in df['year_remod/add']]

# Model 5
df.drop(index=1712, inplace=True)
df.drop(index=1327, inplace=True)

# Model 6
df['before_1940'] = [1 if year < 1940 else 0 for year in df['year_built']]
df['over_10'] = [1 if age > 10 else 0 for age in df['age']]

In [3]:
df.shape

(2049, 88)

In [4]:
test = pd.read_csv('datasets/test.csv', keep_default_na=False, na_values=[''])
test_original = pd.read_csv('datasets/test.csv', keep_default_na=False, na_values=[''])
test = test.rename(str.lower, axis=1)
test.columns = [name.replace(' ', '_') if ' ' in name else name for name in test.columns]


# Model 2
test['age'] = [test['year_built'].max()-year for year in test['year_built']]
test['age*overall_qual'] = test['age']*df['overall_qual']

# Model 3
kitchen_qual_order = {'Po': 0, 'Fa': 1, 'TA': 2, 'Gd': 3, 'Ex': 4}
test['kitchen_qual_order'] = [kitchen_qual_order[n] for n in test['kitchen_qual']]

# Model 4
test['gr_liv_area/totrms_abvgrd'] = test['gr_liv_area']/test['totrms_abvgrd']
test['since_remod/add'] = [2010-year for year in test['year_remod/add']]


# Model 6
test['before_1940'] = [1 if year < 1940 else 0 for year in test['year_built']]
test['over_10'] = [1 if age > 10 else 0 for age in test['age']]


# test # modify as training dataframe
# kaggle_preds = model. predecit(test[features])

In [5]:
test.shape

(878, 87)

In [6]:
features_7 = df[['id', 'gr_liv_area', 'bedroom_abvgr', 'age', 'overall_qual', 'gr_liv_area/totrms_abvgrd', 'full_bath', 'since_remod/add',
       '1st_flr_sf', 'garage_area', 'kitchen_qual_order', 'garage_cars', 'total_bsmt_sf', 'before_1940', 'over_10']]

# features_7 = pd.get_dummies(features_7, columns=['neighborhood'], drop_first=True);

In [7]:
test = test[['id', 'gr_liv_area', 'bedroom_abvgr', 'age', 'overall_qual', 'gr_liv_area/totrms_abvgrd', 'full_bath', 'since_remod/add',
       '1st_flr_sf', 'garage_area', 'kitchen_qual_order', 'garage_cars', 'total_bsmt_sf', 'before_1940', 'over_10']]

In [8]:
features_7.shape

(2049, 15)

In [9]:
test.shape

(878, 15)

In [10]:
X = features_7
y = df['saleprice']
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
pf = PolynomialFeatures(include_bias=False)
pf.fit(X_train)
X_train_pf = pf.transform(X_train)
X_test_pf = pf.transform(X_test)
pd.DataFrame(X_test_pf, columns=pf.get_feature_names(X.columns));

In [11]:
ss = StandardScaler()                  
ss.fit(X_train_pf)
Z_train = ss.transform(X_train_pf)
Z_test = ss.transform(X_test_pf)

In [12]:
Z_train.shape

(1536, 135)

In [13]:
test = pf.transform(test)
test = ss.fit_transform (test)
test.shape

(878, 135)

In [14]:
Z_train.shape

(1536, 135)

In [15]:
l_alphas = np.linspace(0.000001, .3, 3000)
lcv = LassoCV()
lcv.fit(Z_train, y_train)
lcv.score(Z_train, y_train), lcv.score(Z_test, y_test)

C:\Users\mmoli\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1409516028.005005, tolerance: 770758974.6212007
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\mmoli\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1086244092.3481445, tolerance: 770758974.6212007
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\mmoli\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1099703137.3876953, tolerance: 770758974.6212007
  model = cd_fast.enet_coordinate_descent_gram(
C:\Users\mmoli\anaconda3\lib\site-packages\sklearn\linear_model\_coordinate_descent.py:526: 

(0.9014166273617451, 0.8866754800932498)

In [16]:
lcv.alpha_

65.35377948965409

In [17]:
lcv.score

<bound method RegressorMixin.score of LassoCV()>

In [18]:
# lcv.fit(Z_train, y_train)
# y_preds = lcv.predict(Z_test)
# resids = y_test-y_preds                     
# resids.mean()

In [19]:
df.shape

(2049, 88)

In [20]:
test.shape

(878, 135)

In [21]:
predicted_prices = lcv.predict(test)

In [22]:
predicted_prices.shape

(878,)

In [24]:
# test['Id']

In [26]:
sub_2 = pd.DataFrame({'Id': test_original['Id'].values , 'SalePrice': predicted_prices})

In [27]:
sub_2.to_csv('submission.csv', index=False)